# Importing the libraries

In [1]:
!pip3 install scikit-image



[notice] A new release of pip available: 22.3.1 -> 23.1.2
[notice] To update, run: pip3 install --upgrade pip


In [2]:
import os
import pandas as pd
import numpy as np
import json
import spotipy
import spotipy.oauth2 as oauth2
from spotipy.oauth2 import SpotifyOAuth,SpotifyClientCredentials
import yaml
import re
from tqdm import tqdm
import multiprocessing as mp
import time
import random
import datetime
import pickle
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
from skimage import io
from sklearn.preprocessing import OneHotEncoder

In [3]:
stream= open("spotify/spotify.yaml")
spotify_details = yaml.safe_load(stream)
auth_manager = SpotifyClientCredentials(client_id=spotify_details['Client_id'],
                                        client_secret=spotify_details['client_secret'])
sp = spotipy.client.Spotify(auth_manager=auth_manager)

# Importing the dataset

In [4]:
dtypes = {'track_uri': 'object', 'artist_uri': 'object', 'album_uri': 'object', 'danceability': 'float16', 'energy': 'float16', 'key': 'float16',
               'loudness': 'float16', 'mode': 'float16', 'speechiness': 'float16', 'acousticness': 'float16', 'instrumentalness': 'float16',
               'liveness': 'float16', 'valence': 'float16', 'tempo': 'float16', 'duration_ms': 'float32', 'time_signature': 'float16',
               'Track_release_date': 'int8', 'Track_pop': 'float16', 'Artist_pop': 'float16', 'Artist_genres': 'object'}
try:
    df=pd.read_csv('Data/1M_unique_processed_data_grow.csv',dtype=dtypes)
except:
    print('Failed to load grow')
    df=pd.read_csv('Data/1M_unique_processed_data.csv',dtype=dtypes)
    print("Done")


Failed to load grow
Done


In [5]:
df.shape

(300528, 20)

In [6]:
df.head()

,track_uri,artist_uri,album_uri,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,Track_release_date,Track_pop,Artist_pop,Artist_genres
0,3uvsVUrAaGQJCTEUR1S3Sx,0ypTT9UqAU5sZpPo5JZmjR,2kIoMj5Ht14l2PnNRa2abC,0.522949,0.422119,3.0,-10.382812,0.0,0.026993,0.783203,0.005138,0.089783,0.145020,129.875,242060.0,4.0,40,0.000000,7.000000,british_singer-songwriter
1,3uvsVUrAaGQJCTEUR1S3Sx,0ypTT9UqAU5sZpPo5JZmjR,2kIoMj5Ht14l2PnNRa2abC,0.522949,0.422119,3.0,-10.382812,0.0,0.026993,0.783203,0.005138,0.089783,0.145020,129.875,242060.0,4.0,40,0.000000,7.000000,british_singer-songwriter
2,4JxNnZ2wf9t3KGnB0cIwRU,0ypTT9UqAU5sZpPo5JZmjR,0CMDyjTSb2N8kpk3fSbiUp,0.523926,0.341064,9.0,-11.421875,0.0,0.031204,0.570801,0.281982,0.106018,0.295898,80.000,267123.0,4.0,40,0.000000,7.000000,british_singer-songwriter
3,74Iy3ovRFMr5erVPha9SAa,0ypTT9UqAU5sZpPo5JZmjR,0gbEBO45KjT4pAlB83NRu4,0.420898,0.504883,6.0,-9.437500,0.0,0.044006,0.046692,0.001550,0.101013,0.131958,134.875,231272.0,4.0,40,0.000000,7.000000,british_singer-songwriter
4,0heE5tAAaDQmnGhVDImPl2,0h3YCmvRJ2jqt4jFiR6nGL,3g7TTE6375PGIBsM9Tlk9I,0.492920,0.594238,8.0,-4.402344,0.0,0.037811,0.256104,0.000000,0.075928,0.303955,123.750,225515.0,4.0,40,12.601562,10.601562,norwegian_pop


# Test

Extract playlist tracks and artist uri

In [7]:
def get_IDs (user, playlist_id):
    track_ids = []
    artist_id = []
    playlist=sp.user_playlist (user, playlist_id)
    for item in playlist['tracks']['items']:
        track=item['track']
        track_ids.append(track['id'])
        artist=item['track']['artists']
        artist_id.append(artist[0]['id'])
    return track_ids,artist_id


track_ids,artist_id = get_IDs ('Ruby', 'spotify:playlist:37i9dQZF1DX8FwnYE6PRvL') 
print (len(track_ids))
print (len(artist_id))

100
100


getting the unique URI and repeating the extraction features and preprocessing steps for the user's playlist (input)

In [8]:
artist_id_uni=list(set(artist_id))
track_ids_uni=list(set(track_ids))

In [9]:
df.shape

(300528, 20)

In [10]:
audio_features=pd.DataFrame()
for i in tqdm(range(0,len(track_ids_uni),25)):
    try:
        track_feature = sp.audio_features(track_ids_uni[i:i+25])
        track_df = pd.DataFrame(track_feature)
        audio_features=pd.concat([audio_features,track_df],axis=0)
    except Exception as e:
        print(e)
        continue

100%|█████████████████████████████████████████████| 4/4 [00:00<00:00,  5.23it/s]


In [11]:
track_=pd.DataFrame()
for i in tqdm(range(0,len(track_ids_uni),25)):
    try:
        track_features = sp.tracks(track_ids_uni[i:i+25])
        for x in range(25):
            track_pop=pd.DataFrame([track_ids_uni[i+x]],columns=['Track_uri'])
            track_pop['Track_release_date']=track_features['tracks'][x]['album']['release_date']
            track_pop['Track_pop'] = track_features['tracks'][x]["popularity"]
            track_pop['Artist_uri']=track_features['tracks'][x]['artists'][0]['id']
            track_pop['Album_uri']=track_features['tracks'][x]['album']['id']
            track_=pd.concat([track_,track_pop],axis=0)
    except Exception as e:
        print(e)
        continue

100%|█████████████████████████████████████████████| 4/4 [00:01<00:00,  2.66it/s]


In [12]:
artist_=pd.DataFrame()
for i in tqdm(range(0,len(artist_id_uni),25)):
    try:
        artist_features = sp.artists(artist_id_uni[i:i+25])
        for x in range(25):
            artist_df=pd.DataFrame([artist_id_uni[i+x]],columns=['Artist_uri'])
            artist_pop = artist_features['artists'][x]["popularity"]
            artist_genres = artist_features['artists'][x]["genres"]
            artist_df["Artist_pop"] = artist_pop
            if artist_genres: 
                artist_df["genres"] = " ".join([re.sub(' ','_',i) for i in artist_genres])
            else:
                artist_df["genres"] = "unknown"
            artist_=pd.concat([artist_,artist_df],axis=0)
    except Exception as e:
        print(e)
        continue

100%|█████████████████████████████████████████████| 3/3 [00:00<00:00,  3.10it/s]

list index out of range


In [13]:
audio_features.head()

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,0.596,0.605,7,-12.145,1,0.0255,0.1810,0.000331,0.0863,0.886,97.798,audio_features,7e89621JPkKaeDSTQ3avtg,spotify:track:7e89621JPkKaeDSTQ3avtg,https://api.spotify.com/v1/tracks/7e89621JPkKa...,https://api.spotify.com/v1/audio-analysis/7e89...,283800,4
1,0.454,0.913,9,-4.008,1,0.1070,0.4080,0.000271,0.2480,0.521,137.688,audio_features,7ACxUo21jtTHzy7ZEV56vU,spotify:track:7ACxUo21jtTHzy7ZEV56vU,https://api.spotify.com/v1/tracks/7ACxUo21jtTH...,https://api.spotify.com/v1/audio-analysis/7ACx...,293183,4
2,0.502,0.924,4,-6.299,1,0.0503,0.0127,0.000000,0.1620,0.922,149.186,audio_features,1hlveB9M6ijHZRbzZ2teyh,spotify:track:1hlveB9M6ijHZRbzZ2teyh,https://api.spotify.com/v1/tracks/1hlveB9M6ijH...,https://api.spotify.com/v1/audio-analysis/1hlv...,219667,4
3,0.636,0.849,0,-7.075,1,0.1060,0.0438,0.000000,0.3720,0.917,156.036,audio_features,3Pzh926pXggbMe2ZpXyMV7,spotify:track:3Pzh926pXggbMe2ZpXyMV7,https://api.spotify.com/v1/tracks/3Pzh926pXggb...,https://api.spotify.com/v1/audio-analysis/3Pzh...,175493,4
4,0.621,0.901,2,-5.396,1,0.0302,0.1700,0.000000,0.0875,0.744,113.967,audio_features,1Cwsd5xI8CajJz795oy4XF,spotify:track:1Cwsd5xI8CajJz795oy4XF,https://api.spotify.com/v1/tracks/1Cwsd5xI8Caj...,https://api.spotify.com/v1/audio-analysis/1Cws...,300773,4


In [14]:
track_.reset_index(drop=True, inplace=True)

In [15]:
artist_.reset_index(drop=True, inplace=True)

In [16]:
test=pd.DataFrame(track_,columns=['Track_uri','Artist_uri','Album_uri'])

In [17]:
test.reset_index(drop=True, inplace=True)

In [18]:
test

,Track_uri,Artist_uri,Album_uri
0,7e89621JPkKaeDSTQ3avtg,4MVyzYMgTwdP7Z49wAZHx0,54V1ljNtyzAm053oJqi0SH
1,7ACxUo21jtTHzy7ZEV56vU,6ZLTlhejhndI4Rh53vYhrY,4qUMByJ3Pk94BFnCmGaUPS
2,1hlveB9M6ijHZRbzZ2teyh,7b85ve82Sh36a3UAx74wut,0dzqapIToiOhULGvzDKpXm
3,3Pzh926pXggbMe2ZpXyMV7,26T3LtbuGT1Fu9m0eRq5X3,7H814Cg8HV0qpoMheYbhNn
4,1Cwsd5xI8CajJz795oy4XF,0Grjlu7ncIuCaSYvCs9fcd,13btXEnBerpA1UjIVtsMAR
...,...,...,...
95,6EPRKhUOdiFSQwGBRBbvsZ,1DFr97A9HnbV3SKTJFu62M,3rxF05Aux0QTrN533Kjc91
96,6tsojOQ5wHaIjKqIryLZK6,7jy3rLJdDQY21OgRLCZ9sD,1zQ6F8gMagKcPL4SoA80cx
97,3SFXsFpeGmBTtQvKiwYMDA,5LfGQac0EIXyAN8aUwmNAQ,2RNTBrSO8U8XjjEj9RVvZ5
98,70Z9t1qhytWtG4cCmmi7mU,7b85ve82Sh36a3UAx74wut,0dzqapIToiOhULGvzDKpXm


In [19]:
test.rename(columns = {'Track_uri':'track_uri','Artist_uri':'artist_uri','Album_uri':'album_uri'}, inplace = True)

In [20]:
audio_features.drop(columns=['type','uri','track_href','analysis_url'],axis=1,inplace=True)

In [21]:
test.head()

,track_uri,artist_uri,album_uri
0,7e89621JPkKaeDSTQ3avtg,4MVyzYMgTwdP7Z49wAZHx0,54V1ljNtyzAm053oJqi0SH
1,7ACxUo21jtTHzy7ZEV56vU,6ZLTlhejhndI4Rh53vYhrY,4qUMByJ3Pk94BFnCmGaUPS
2,1hlveB9M6ijHZRbzZ2teyh,7b85ve82Sh36a3UAx74wut,0dzqapIToiOhULGvzDKpXm
3,3Pzh926pXggbMe2ZpXyMV7,26T3LtbuGT1Fu9m0eRq5X3,7H814Cg8HV0qpoMheYbhNn
4,1Cwsd5xI8CajJz795oy4XF,0Grjlu7ncIuCaSYvCs9fcd,13btXEnBerpA1UjIVtsMAR


In [22]:
audio_features.head()

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,id,duration_ms,time_signature
0,0.596,0.605,7,-12.145,1,0.0255,0.1810,0.000331,0.0863,0.886,97.798,7e89621JPkKaeDSTQ3avtg,283800,4
1,0.454,0.913,9,-4.008,1,0.1070,0.4080,0.000271,0.2480,0.521,137.688,7ACxUo21jtTHzy7ZEV56vU,293183,4
2,0.502,0.924,4,-6.299,1,0.0503,0.0127,0.000000,0.1620,0.922,149.186,1hlveB9M6ijHZRbzZ2teyh,219667,4
3,0.636,0.849,0,-7.075,1,0.1060,0.0438,0.000000,0.3720,0.917,156.036,3Pzh926pXggbMe2ZpXyMV7,175493,4
4,0.621,0.901,2,-5.396,1,0.0302,0.1700,0.000000,0.0875,0.744,113.967,1Cwsd5xI8CajJz795oy4XF,300773,4


In [23]:
track_.head()

,Track_uri,Track_release_date,Track_pop,Artist_uri,Album_uri
0,7e89621JPkKaeDSTQ3avtg,1974-04-15,83,4MVyzYMgTwdP7Z49wAZHx0,54V1ljNtyzAm053oJqi0SH
1,7ACxUo21jtTHzy7ZEV56vU,2020-09-18,79,6ZLTlhejhndI4Rh53vYhrY,4qUMByJ3Pk94BFnCmGaUPS
2,1hlveB9M6ijHZRbzZ2teyh,1984-05-10,76,7b85ve82Sh36a3UAx74wut,0dzqapIToiOhULGvzDKpXm
3,3Pzh926pXggbMe2ZpXyMV7,2009-03-24,76,26T3LtbuGT1Fu9m0eRq5X3,7H814Cg8HV0qpoMheYbhNn
4,1Cwsd5xI8CajJz795oy4XF,1998-01-01,77,0Grjlu7ncIuCaSYvCs9fcd,13btXEnBerpA1UjIVtsMAR


In [24]:
artist_.head()

,Artist_uri,Artist_pop,genres
0,1w5Kfo2jwwIPruYS2UWh56,75,alternative_metal alternative_rock grunge perm...
1,4UXqAaa6dQYAk18Lv7PEgX,80,emo modern_rock pop pop_punk rock
2,7gP3bB2nilZXLfPHJhMdvc,71,indietronica modern_alternative_rock modern_ro...
3,5gznATMVO85ZcLTkE9ULU7,70,permanent_wave rock
4,0L8ExT028jH3ddEcZwqJJ5,82,alternative_rock funk_metal funk_rock permanen...


In [25]:
test = pd.merge(test,audio_features, left_on = "track_uri", right_on= "id",how = 'outer')
test = pd.merge(test,track_, left_on = "track_uri", right_on= "Track_uri",how = 'outer')
test = pd.merge(test,artist_, left_on = "artist_uri", right_on= "Artist_uri",how = 'outer')

In [26]:
test

,track_uri,artist_uri,album_uri,danceability,energy,key,loudness,mode,speechiness,acousticness,...,duration_ms,time_signature,Track_uri,Track_release_date,Track_pop,Artist_uri_x,Album_uri,Artist_uri_y,Artist_pop,genres
0,7e89621JPkKaeDSTQ3avtg,4MVyzYMgTwdP7Z49wAZHx0,54V1ljNtyzAm053oJqi0SH,0.596,0.605,7,-12.145,1,0.0255,0.18100,...,283800,4,7e89621JPkKaeDSTQ3avtg,1974-04-15,83,4MVyzYMgTwdP7Z49wAZHx0,54V1ljNtyzAm053oJqi0SH,4MVyzYMgTwdP7Z49wAZHx0,73,album_rock classic_rock hard_rock rock souther...
1,7ACxUo21jtTHzy7ZEV56vU,6ZLTlhejhndI4Rh53vYhrY,4qUMByJ3Pk94BFnCmGaUPS,0.454,0.913,9,-4.008,1,0.1070,0.40800,...,293183,4,7ACxUo21jtTHzy7ZEV56vU,2020-09-18,79,6ZLTlhejhndI4Rh53vYhrY,4qUMByJ3Pk94BFnCmGaUPS,6ZLTlhejhndI4Rh53vYhrY,72,album_rock alternative_metal birmingham_metal ...
2,1hlveB9M6ijHZRbzZ2teyh,7b85ve82Sh36a3UAx74wut,0dzqapIToiOhULGvzDKpXm,0.502,0.924,4,-6.299,1,0.0503,0.01270,...,219667,4,1hlveB9M6ijHZRbzZ2teyh,1984-05-10,76,7b85ve82Sh36a3UAx74wut,0dzqapIToiOhULGvzDKpXm,7b85ve82Sh36a3UAx74wut,63,album_rock glam_metal hard_rock metal rock
3,70Z9t1qhytWtG4cCmmi7mU,7b85ve82Sh36a3UAx74wut,0dzqapIToiOhULGvzDKpXm,0.504,0.911,2,-2.970,1,0.0738,0.00512,...,179760,4,70Z9t1qhytWtG4cCmmi7mU,1984-05-10,74,7b85ve82Sh36a3UAx74wut,0dzqapIToiOhULGvzDKpXm,7b85ve82Sh36a3UAx74wut,63,album_rock glam_metal hard_rock metal rock
4,3Pzh926pXggbMe2ZpXyMV7,26T3LtbuGT1Fu9m0eRq5X3,7H814Cg8HV0qpoMheYbhNn,0.636,0.849,0,-7.075,1,0.1060,0.04380,...,175493,4,3Pzh926pXggbMe2ZpXyMV7,2009-03-24,76,26T3LtbuGT1Fu9m0eRq5X3,7H814Cg8HV0qpoMheYbhNn,26T3LtbuGT1Fu9m0eRq5X3,74,modern_alternative_rock modern_rock pov:_indie...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,3EYOJ48Et32uATr9ZmLnAo,5NGO30tJxFlKixkPSgXcFE,1H9g6j4Wwj6wh6p8YHVtkf,0.618,0.748,0,-8.905,1,0.3580,0.03840,...,191947,4,3EYOJ48Et32uATr9ZmLnAo,1978-11-02,81,5NGO30tJxFlKixkPSgXcFE,1H9g6j4Wwj6wh6p8YHVtkf,5NGO30tJxFlKixkPSgXcFE,76,album_rock classic_rock permanent_wave rock
96,58XWGx7KNNkKneHdprcprX,27T030eWyCQRmDyuvr1kxY,71cfSO0iO1fjgQLEb3Wc6C,0.482,0.617,4,-12.931,0,0.0458,0.00543,...,252493,4,58XWGx7KNNkKneHdprcprX,1984-03-27,74,27T030eWyCQRmDyuvr1kxY,71cfSO0iO1fjgQLEb3Wc6C,27T030eWyCQRmDyuvr1kxY,73,album_rock german_hard_rock german_metal germa...
97,6QewNVIDKdSl8Y3ycuHIei,1w5Kfo2jwwIPruYS2UWh56,5B4PYA7wNN4WdEXdIJu58a,0.394,0.944,10,-6.506,0,0.0761,0.00178,...,292581,4,6QewNVIDKdSl8Y3ycuHIei,1991-08-27,80,1w5Kfo2jwwIPruYS2UWh56,5B4PYA7wNN4WdEXdIJu58a,1w5Kfo2jwwIPruYS2UWh56,75,alternative_metal alternative_rock grunge perm...
98,6EPRKhUOdiFSQwGBRBbvsZ,1DFr97A9HnbV3SKTJFu62M,3rxF05Aux0QTrN533Kjc91,0.342,0.910,4,-7.748,1,0.0758,0.00004,...,166360,4,6EPRKhUOdiFSQwGBRBbvsZ,1980-11-08,76,1DFr97A9HnbV3SKTJFu62M,3rxF05Aux0QTrN533Kjc91,1DFr97A9HnbV3SKTJFu62M,66,alternative_metal hard_rock metal nu_metal roc...


In [27]:
# del audio_features,track_,artist_

In [28]:
test.columns

Index(['track_uri', 'artist_uri', 'album_uri', 'danceability', 'energy', 'key',
       'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness',
       'liveness', 'valence', 'tempo', 'id', 'duration_ms', 'time_signature',
       'Track_uri', 'Track_release_date', 'Track_pop', 'Artist_uri_x',
       'Album_uri', 'Artist_uri_y', 'Artist_pop', 'genres'],
      dtype='object')

In [29]:
test.rename(columns = {'genres':'Artist_genres'}, inplace = True)

In [30]:
test.drop(columns=['Track_uri','Artist_uri_x','Artist_uri_y','Album_uri','id'],axis=1,inplace=True)

In [31]:
test.isnull().sum()

track_uri             0
artist_uri            0
album_uri             0
danceability          0
energy                0
key                   0
loudness              0
mode                  0
speechiness           0
acousticness          0
instrumentalness      0
liveness              0
valence               0
tempo                 0
duration_ms           0
time_signature        0
Track_release_date    0
Track_pop             0
Artist_pop            0
Artist_genres         0
dtype: int64

In [32]:
test.dropna(axis=0,inplace=True)

In [33]:
test.shape

(100, 20)

In [34]:
test.columns

Index(['track_uri', 'artist_uri', 'album_uri', 'danceability', 'energy', 'key',
       'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness',
       'liveness', 'valence', 'tempo', 'duration_ms', 'time_signature',
       'Track_release_date', 'Track_pop', 'Artist_pop', 'Artist_genres'],
      dtype='object')

In [35]:
test.head()

,track_uri,artist_uri,album_uri,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,Track_release_date,Track_pop,Artist_pop,Artist_genres
0,7e89621JPkKaeDSTQ3avtg,4MVyzYMgTwdP7Z49wAZHx0,54V1ljNtyzAm053oJqi0SH,0.596,0.605,7,-12.145,1,0.0255,0.18100,0.000331,0.0863,0.886,97.798,283800,4,1974-04-15,83,73,album_rock classic_rock hard_rock rock souther...
1,7ACxUo21jtTHzy7ZEV56vU,6ZLTlhejhndI4Rh53vYhrY,4qUMByJ3Pk94BFnCmGaUPS,0.454,0.913,9,-4.008,1,0.1070,0.40800,0.000271,0.2480,0.521,137.688,293183,4,2020-09-18,79,72,album_rock alternative_metal birmingham_metal ...
2,1hlveB9M6ijHZRbzZ2teyh,7b85ve82Sh36a3UAx74wut,0dzqapIToiOhULGvzDKpXm,0.502,0.924,4,-6.299,1,0.0503,0.01270,0.000000,0.1620,0.922,149.186,219667,4,1984-05-10,76,63,album_rock glam_metal hard_rock metal rock
3,70Z9t1qhytWtG4cCmmi7mU,7b85ve82Sh36a3UAx74wut,0dzqapIToiOhULGvzDKpXm,0.504,0.911,2,-2.970,1,0.0738,0.00512,0.012300,0.3550,0.671,106.244,179760,4,1984-05-10,74,63,album_rock glam_metal hard_rock metal rock
4,3Pzh926pXggbMe2ZpXyMV7,26T3LtbuGT1Fu9m0eRq5X3,7H814Cg8HV0qpoMheYbhNn,0.636,0.849,0,-7.075,1,0.1060,0.04380,0.000000,0.3720,0.917,156.036,175493,4,2009-03-24,76,74,modern_alternative_rock modern_rock pov:_indie...


In [36]:
test['Track_pop'] = test['Track_pop'].apply(lambda x: int(x/5))
test['Artist_pop'] = test['Artist_pop'].apply(lambda x: int(x/5))
test['Track_release_date'] = test['Track_release_date'].apply(lambda x: x.split('-')[0])
test['Track_release_date']=test['Track_release_date'].astype('int16')
test['Track_release_date'] = test['Track_release_date'].apply(lambda x: int(x/50))

In [37]:
test.head()

,track_uri,artist_uri,album_uri,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,Track_release_date,Track_pop,Artist_pop,Artist_genres
0,7e89621JPkKaeDSTQ3avtg,4MVyzYMgTwdP7Z49wAZHx0,54V1ljNtyzAm053oJqi0SH,0.596,0.605,7,-12.145,1,0.0255,0.18100,0.000331,0.0863,0.886,97.798,283800,4,39,16,14,album_rock classic_rock hard_rock rock souther...
1,7ACxUo21jtTHzy7ZEV56vU,6ZLTlhejhndI4Rh53vYhrY,4qUMByJ3Pk94BFnCmGaUPS,0.454,0.913,9,-4.008,1,0.1070,0.40800,0.000271,0.2480,0.521,137.688,293183,4,40,15,14,album_rock alternative_metal birmingham_metal ...
2,1hlveB9M6ijHZRbzZ2teyh,7b85ve82Sh36a3UAx74wut,0dzqapIToiOhULGvzDKpXm,0.502,0.924,4,-6.299,1,0.0503,0.01270,0.000000,0.1620,0.922,149.186,219667,4,39,15,12,album_rock glam_metal hard_rock metal rock
3,70Z9t1qhytWtG4cCmmi7mU,7b85ve82Sh36a3UAx74wut,0dzqapIToiOhULGvzDKpXm,0.504,0.911,2,-2.970,1,0.0738,0.00512,0.012300,0.3550,0.671,106.244,179760,4,39,14,12,album_rock glam_metal hard_rock metal rock
4,3Pzh926pXggbMe2ZpXyMV7,26T3LtbuGT1Fu9m0eRq5X3,7H814Cg8HV0qpoMheYbhNn,0.636,0.849,0,-7.075,1,0.1060,0.04380,0.000000,0.3720,0.917,156.036,175493,4,40,15,14,modern_alternative_rock modern_rock pov:_indie...


In [38]:
test[['danceability', 'energy', 'key','loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness','liveness', 'valence', 'tempo', 'time_signature']]=test[['danceability', 'energy', 'key','loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness','liveness', 'valence', 'tempo','time_signature']].astype('float16')
test[['duration_ms']]=test[['duration_ms']].astype('float32')
test[['Track_release_date', 'Track_pop', 'Artist_pop']]=test[['Track_release_date', 'Track_pop', 'Artist_pop']].astype('int8')


In [39]:
test.shape

(100, 20)

In [40]:
currentdf=len(df)

In [41]:
df.shape

(300528, 20)

In [42]:
currentdf

300528

In [43]:
df=pd.concat([df,test],axis=0)

In [44]:
df.shape

(300628, 20)

In [45]:
df[df.duplicated(subset=['track_uri'])]

,track_uri,artist_uri,album_uri,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,Track_release_date,Track_pop,Artist_pop,Artist_genres
1,3uvsVUrAaGQJCTEUR1S3Sx,0ypTT9UqAU5sZpPo5JZmjR,2kIoMj5Ht14l2PnNRa2abC,0.522949,0.422119,3.0,-10.382812,0.0,0.026993,0.783203,0.005138,0.089783,0.145020,129.8750,242060.0,4.0,40,0.000000,7.000000,british_singer-songwriter
5,0heE5tAAaDQmnGhVDImPl2,0h3YCmvRJ2jqt4jFiR6nGL,3g7TTE6375PGIBsM9Tlk9I,0.492920,0.594238,8.0,-4.402344,0.0,0.037811,0.256104,0.000000,0.075928,0.303955,123.7500,225515.0,4.0,40,12.601562,10.601562,norwegian_pop
8,3omXshBamrREltcf24gYDC,6VDdCwrBM4qQaGxoAyxyJC,2H09itV5a5yUcGyk9u9HwY,0.468018,0.691895,2.0,-4.015625,1.0,0.029495,0.020203,0.000002,0.522949,0.561035,78.0000,200360.0,4.0,40,13.601562,11.796875,indie_rock modern_alternative_rock modern_rock...
71,6TYWE19e35N7Bn5heHwyY6,4uNv6RD2YXwoaKgHfJZkkL,38y7zXf95O9Afh7ZXIoyq1,0.553223,0.563965,9.0,-7.070312,0.0,0.041809,0.039001,0.000000,0.318115,0.310059,150.0000,232643.0,4.0,40,6.199219,12.203125,indie_electropop indie_poptimism
76,1xznGGDReH1oQq0xzbwXa3,3TVXtAsR1Inumwj472S9r4,3hARKC8cinq3mZLLAEaBh9,0.791016,0.619141,1.0,-5.886719,1.0,0.053192,0.007843,0.004230,0.351074,0.371094,104.0000,173987.0,4.0,40,0.199951,19.203125,canadian_hip_hop canadian_pop hip_hop rap toro...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
92,3MrRksHupTVEQ7YbA0FsZK,7Js6Lde8thlIHXggv2SCBz,5Jkd47JEaCU1g4DcGBnHm3,0.528809,0.842773,6.0,-6.906250,0.0,0.061615,0.062805,0.000827,0.060791,0.187988,117.6875,310333.0,4.0,39,15.000000,13.000000,album_rock glam_metal hard_rock rock swedish_h...
93,2aoo2jlRnM3A0NyLQqMN2f,776Uo845nYHJpNaStv1Ds4,5z090LQztiqh13wYspQvKQ,0.437988,0.805176,8.0,-6.238281,1.0,0.062408,0.002550,0.000057,0.084229,0.563965,113.2500,240800.0,4.0,39,15.000000,14.000000,acid_rock album_rock classic_rock hard_rock pr...
97,6QewNVIDKdSl8Y3ycuHIei,1w5Kfo2jwwIPruYS2UWh56,5B4PYA7wNN4WdEXdIJu58a,0.394043,0.943848,10.0,-6.507812,0.0,0.076111,0.001780,0.000005,0.298096,0.264893,105.3125,292581.0,4.0,39,16.000000,15.000000,alternative_metal alternative_rock grunge perm...
98,6EPRKhUOdiFSQwGBRBbvsZ,1DFr97A9HnbV3SKTJFu62M,3rxF05Aux0QTrN533Kjc91,0.342041,0.910156,4.0,-7.746094,1.0,0.075806,0.000040,0.000121,0.082275,0.546875,140.5000,166360.0,4.0,39,15.000000,13.000000,alternative_metal hard_rock metal nu_metal roc...


In [46]:
# df.drop_duplicates(subset=['track_uri'],inplace=True,keep='last') ## keep last to keep the dataset updated 

In [47]:
df.isnull().sum()

track_uri             0
artist_uri            0
album_uri             0
danceability          0
energy                0
key                   0
loudness              0
mode                  0
speechiness           0
acousticness          0
instrumentalness      0
liveness              0
valence               0
tempo                 0
duration_ms           0
time_signature        0
Track_release_date    0
Track_pop             0
Artist_pop            0
Artist_genres         0
dtype: int64

In [48]:
df.isnull().sum()

track_uri             0
artist_uri            0
album_uri             0
danceability          0
energy                0
key                   0
loudness              0
mode                  0
speechiness           0
acousticness          0
instrumentalness      0
liveness              0
valence               0
tempo                 0
duration_ms           0
time_signature        0
Track_release_date    0
Track_pop             0
Artist_pop            0
Artist_genres         0
dtype: int64

In [49]:
# df.drop(['track_uri', 'artist_uri'], axis='columns', inplace = True)

In [50]:
df.shape

(300628, 20)

In [51]:
df.dropna(axis=0).shape

(300628, 20)

In [52]:
print('{} New Tracks Found'.format(len(df)-currentdf))

100 New Tracks Found


In [53]:
df

,track_uri,artist_uri,album_uri,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,Track_release_date,Track_pop,Artist_pop,Artist_genres
0,3uvsVUrAaGQJCTEUR1S3Sx,0ypTT9UqAU5sZpPo5JZmjR,2kIoMj5Ht14l2PnNRa2abC,0.522949,0.422119,3.0,-10.382812,0.0,0.026993,0.783203,0.005138,0.089783,0.145020,129.8750,242060.0,4.0,40,0.000000,7.000000,british_singer-songwriter
1,3uvsVUrAaGQJCTEUR1S3Sx,0ypTT9UqAU5sZpPo5JZmjR,2kIoMj5Ht14l2PnNRa2abC,0.522949,0.422119,3.0,-10.382812,0.0,0.026993,0.783203,0.005138,0.089783,0.145020,129.8750,242060.0,4.0,40,0.000000,7.000000,british_singer-songwriter
2,4JxNnZ2wf9t3KGnB0cIwRU,0ypTT9UqAU5sZpPo5JZmjR,0CMDyjTSb2N8kpk3fSbiUp,0.523926,0.341064,9.0,-11.421875,0.0,0.031204,0.570801,0.281982,0.106018,0.295898,80.0000,267123.0,4.0,40,0.000000,7.000000,british_singer-songwriter
3,74Iy3ovRFMr5erVPha9SAa,0ypTT9UqAU5sZpPo5JZmjR,0gbEBO45KjT4pAlB83NRu4,0.420898,0.504883,6.0,-9.437500,0.0,0.044006,0.046692,0.001550,0.101013,0.131958,134.8750,231272.0,4.0,40,0.000000,7.000000,british_singer-songwriter
4,0heE5tAAaDQmnGhVDImPl2,0h3YCmvRJ2jqt4jFiR6nGL,3g7TTE6375PGIBsM9Tlk9I,0.492920,0.594238,8.0,-4.402344,0.0,0.037811,0.256104,0.000000,0.075928,0.303955,123.7500,225515.0,4.0,40,12.601562,10.601562,norwegian_pop
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,3EYOJ48Et32uATr9ZmLnAo,5NGO30tJxFlKixkPSgXcFE,1H9g6j4Wwj6wh6p8YHVtkf,0.618164,0.748047,0.0,-8.906250,1.0,0.357910,0.038391,0.000004,0.047302,0.663086,134.1250,191947.0,4.0,39,16.000000,15.000000,album_rock classic_rock permanent_wave rock
96,58XWGx7KNNkKneHdprcprX,27T030eWyCQRmDyuvr1kxY,71cfSO0iO1fjgQLEb3Wc6C,0.481934,0.617188,4.0,-12.929688,0.0,0.045807,0.005428,0.000146,0.283936,0.717773,125.6875,252493.0,4.0,39,14.000000,14.000000,album_rock german_hard_rock german_metal germa...
97,6QewNVIDKdSl8Y3ycuHIei,1w5Kfo2jwwIPruYS2UWh56,5B4PYA7wNN4WdEXdIJu58a,0.394043,0.943848,10.0,-6.507812,0.0,0.076111,0.001780,0.000005,0.298096,0.264893,105.3125,292581.0,4.0,39,16.000000,15.000000,alternative_metal alternative_rock grunge perm...
98,6EPRKhUOdiFSQwGBRBbvsZ,1DFr97A9HnbV3SKTJFu62M,3rxF05Aux0QTrN533Kjc91,0.342041,0.910156,4.0,-7.746094,1.0,0.075806,0.000040,0.000121,0.082275,0.546875,140.5000,166360.0,4.0,39,15.000000,13.000000,alternative_metal hard_rock metal nu_metal roc...


In [54]:
#saving the tracks if they weren't found in the dataset
if len(df)>currentdf:   
    df.to_csv('data/1M_unique_processed_data_grow.csv',index=False)
    print('{} New Found'.format(len(df)-currentdf))
    streamlit=df[df.Track_pop >0]             # dropped track with 0 popularity score to save space and ram for the final model
    streamlit.to_csv('data/streamlit.csv',index=False)
    del streamlit

100 New Found


In [55]:
df = df[~df['track_uri'].isin(test['track_uri'].values)]

In [56]:
test.head()

,track_uri,artist_uri,album_uri,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,Track_release_date,Track_pop,Artist_pop,Artist_genres
0,7e89621JPkKaeDSTQ3avtg,4MVyzYMgTwdP7Z49wAZHx0,54V1ljNtyzAm053oJqi0SH,0.596191,0.604980,7.0,-12.148438,1.0,0.025497,0.181030,0.000331,0.086304,0.886230,97.8125,283800.0,4.0,39,16,14,album_rock classic_rock hard_rock rock souther...
1,7ACxUo21jtTHzy7ZEV56vU,6ZLTlhejhndI4Rh53vYhrY,4qUMByJ3Pk94BFnCmGaUPS,0.454102,0.913086,9.0,-4.007812,1.0,0.106995,0.407959,0.000271,0.248047,0.520996,137.7500,293183.0,4.0,40,15,14,album_rock alternative_metal birmingham_metal ...
2,1hlveB9M6ijHZRbzZ2teyh,7b85ve82Sh36a3UAx74wut,0dzqapIToiOhULGvzDKpXm,0.501953,0.923828,4.0,-6.300781,1.0,0.050293,0.012703,0.000000,0.161987,0.921875,149.1250,219667.0,4.0,39,15,12,album_rock glam_metal hard_rock metal rock
3,70Z9t1qhytWtG4cCmmi7mU,7b85ve82Sh36a3UAx74wut,0dzqapIToiOhULGvzDKpXm,0.503906,0.911133,2.0,-2.970703,1.0,0.073792,0.005119,0.012299,0.354980,0.670898,106.2500,179760.0,4.0,39,14,12,album_rock glam_metal hard_rock metal rock
4,3Pzh926pXggbMe2ZpXyMV7,26T3LtbuGT1Fu9m0eRq5X3,7H814Cg8HV0qpoMheYbhNn,0.636230,0.849121,0.0,-7.074219,1.0,0.106018,0.043793,0.000000,0.372070,0.916992,156.0000,175493.0,4.0,40,15,14,modern_alternative_rock modern_rock pov:_indie...


In [57]:
test['Artist_genres'] = test['Artist_genres'].apply(lambda x: x.split(" "))

In [58]:
test.head()

,track_uri,artist_uri,album_uri,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,Track_release_date,Track_pop,Artist_pop,Artist_genres
0,7e89621JPkKaeDSTQ3avtg,4MVyzYMgTwdP7Z49wAZHx0,54V1ljNtyzAm053oJqi0SH,0.596191,0.604980,7.0,-12.148438,1.0,0.025497,0.181030,0.000331,0.086304,0.886230,97.8125,283800.0,4.0,39,16,14,"[album_rock, classic_rock, hard_rock, rock, so..."
1,7ACxUo21jtTHzy7ZEV56vU,6ZLTlhejhndI4Rh53vYhrY,4qUMByJ3Pk94BFnCmGaUPS,0.454102,0.913086,9.0,-4.007812,1.0,0.106995,0.407959,0.000271,0.248047,0.520996,137.7500,293183.0,4.0,40,15,14,"[album_rock, alternative_metal, birmingham_met..."
2,1hlveB9M6ijHZRbzZ2teyh,7b85ve82Sh36a3UAx74wut,0dzqapIToiOhULGvzDKpXm,0.501953,0.923828,4.0,-6.300781,1.0,0.050293,0.012703,0.000000,0.161987,0.921875,149.1250,219667.0,4.0,39,15,12,"[album_rock, glam_metal, hard_rock, metal, rock]"
3,70Z9t1qhytWtG4cCmmi7mU,7b85ve82Sh36a3UAx74wut,0dzqapIToiOhULGvzDKpXm,0.503906,0.911133,2.0,-2.970703,1.0,0.073792,0.005119,0.012299,0.354980,0.670898,106.2500,179760.0,4.0,39,14,12,"[album_rock, glam_metal, hard_rock, metal, rock]"
4,3Pzh926pXggbMe2ZpXyMV7,26T3LtbuGT1Fu9m0eRq5X3,7H814Cg8HV0qpoMheYbhNn,0.636230,0.849121,0.0,-7.074219,1.0,0.106018,0.043793,0.000000,0.372070,0.916992,156.0000,175493.0,4.0,40,15,14,"[modern_alternative_rock, modern_rock, pov:_in..."


In [59]:
tfidf = TfidfVectorizer(max_features=3) #max_features=5 
tfidf_matrix = tfidf.fit_transform(test['Artist_genres'].apply(lambda x: " ".join(x)))
genre_df = pd.DataFrame(tfidf_matrix.toarray())
genre_df.columns = ['genre' + "|" + i for i in tfidf.get_feature_names_out()]

In [60]:
genre_df.head()

,genre|album_rock,genre|hard_rock,genre|rock
0,0.694074,0.635045,0.339087
1,0.694074,0.635045,0.339087
2,0.694074,0.635045,0.339087
3,0.694074,0.635045,0.339087
4,0.000000,0.000000,1.000000


In [61]:
genre_df=genre_df.astype('float16')

In [62]:
test.drop(columns=['Artist_genres'],axis=1,inplace=True)

In [63]:
test = pd.concat([test.reset_index(drop=True), genre_df.reset_index(drop=True)],axis = 1)


In [64]:
test.isna().sum().sum()

0

In [65]:
df.shape

(300465, 20)

In [66]:
df.head()

,track_uri,artist_uri,album_uri,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,Track_release_date,Track_pop,Artist_pop,Artist_genres
0,3uvsVUrAaGQJCTEUR1S3Sx,0ypTT9UqAU5sZpPo5JZmjR,2kIoMj5Ht14l2PnNRa2abC,0.522949,0.422119,3.0,-10.382812,0.0,0.026993,0.783203,0.005138,0.089783,0.145020,129.875,242060.0,4.0,40,0.000000,7.000000,british_singer-songwriter
1,3uvsVUrAaGQJCTEUR1S3Sx,0ypTT9UqAU5sZpPo5JZmjR,2kIoMj5Ht14l2PnNRa2abC,0.522949,0.422119,3.0,-10.382812,0.0,0.026993,0.783203,0.005138,0.089783,0.145020,129.875,242060.0,4.0,40,0.000000,7.000000,british_singer-songwriter
2,4JxNnZ2wf9t3KGnB0cIwRU,0ypTT9UqAU5sZpPo5JZmjR,0CMDyjTSb2N8kpk3fSbiUp,0.523926,0.341064,9.0,-11.421875,0.0,0.031204,0.570801,0.281982,0.106018,0.295898,80.000,267123.0,4.0,40,0.000000,7.000000,british_singer-songwriter
3,74Iy3ovRFMr5erVPha9SAa,0ypTT9UqAU5sZpPo5JZmjR,0gbEBO45KjT4pAlB83NRu4,0.420898,0.504883,6.0,-9.437500,0.0,0.044006,0.046692,0.001550,0.101013,0.131958,134.875,231272.0,4.0,40,0.000000,7.000000,british_singer-songwriter
4,0heE5tAAaDQmnGhVDImPl2,0h3YCmvRJ2jqt4jFiR6nGL,3g7TTE6375PGIBsM9Tlk9I,0.492920,0.594238,8.0,-4.402344,0.0,0.037811,0.256104,0.000000,0.075928,0.303955,123.750,225515.0,4.0,40,12.601562,10.601562,norwegian_pop


# df

In [67]:
df['Artist_genres'] = df['Artist_genres'].apply(lambda x: x.split(" "))
tfidf_matrix = tfidf.transform(df['Artist_genres'].apply(lambda x: " ".join(x)))
genre_df = pd.DataFrame(tfidf_matrix.toarray())
genre_df.columns = ['genre' + "|" + i for i in tfidf.get_feature_names_out()]

/var/folders/cm/77wd3wlj2vq84sf8r4wmcm0h0000gn/T/ipykernel_46664/3430617882.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Artist_genres'] = df['Artist_genres'].apply(lambda x: x.split(" "))


In [68]:
genre_df=genre_df.astype('float16')

In [69]:
df.drop(columns=['Artist_genres'],axis=1,inplace=True)

/var/folders/cm/77wd3wlj2vq84sf8r4wmcm0h0000gn/T/ipykernel_46664/2266508896.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(columns=['Artist_genres'],axis=1,inplace=True)


In [70]:
df = pd.concat([df.reset_index(drop=True), genre_df.reset_index(drop=True)],axis = 1)

# pred

In [71]:
try:
    df.drop(columns=['genre|unknown'],axis=1,inplace=True)
    test.drop(columns=['genre|unknown'],axis=1,inplace=True)
except:
    print('genre|unknown not found')

genre|unknown not found


In [72]:
test.columns

Index(['track_uri', 'artist_uri', 'album_uri', 'danceability', 'energy', 'key',
       'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness',
       'liveness', 'valence', 'tempo', 'duration_ms', 'time_signature',
       'Track_release_date', 'Track_pop', 'Artist_pop', 'genre|album_rock',
       'genre|hard_rock', 'genre|rock'],
      dtype='object')

In [73]:
df.columns

Index(['track_uri', 'artist_uri', 'album_uri', 'danceability', 'energy', 'key',
       'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness',
       'liveness', 'valence', 'tempo', 'duration_ms', 'time_signature',
       'Track_release_date', 'Track_pop', 'Artist_pop', 'genre|album_rock',
       'genre|hard_rock', 'genre|rock'],
      dtype='object')

I was first using OneHotEncoder for "Track_release_date", "Track_pop", and "Artist_pop," but I found no difference in the final result other than high memory usage.

In [74]:
""" ohe = OneHotEncoder(handle_unknown='ignore')
dummies = pd.DataFrame(ohe.fit_transform(test[['Track_release_date', 'Track_pop', 'Artist_pop']]).toarray(), index=test.index,dtype=int)
column_name = ohe.get_feature_names_out(['Track_release_date', 'Track_pop', 'Artist_pop'])
dummies.columns=column_name
test = pd.concat([test.drop(['Track_release_date', 'Track_pop', 'Artist_pop'], axis=1), dummies], axis=1) """

" ohe = OneHotEncoder(handle_unknown='ignore')\ndummies = pd.DataFrame(ohe.fit_transform(test[['Track_release_date', 'Track_pop', 'Artist_pop']]).toarray(), index=test.index,dtype=int)\ncolumn_name = ohe.get_feature_names_out(['Track_release_date', 'Track_pop', 'Artist_pop'])\ndummies.columns=column_name\ntest = pd.concat([test.drop(['Track_release_date', 'Track_pop', 'Artist_pop'], axis=1), dummies], axis=1) "

In [76]:
""" ohe2 = OneHotEncoder(categories=ohe.categories_,handle_unknown='ignore')
dummies = pd.DataFrame(ohe2.fit_transform(df[['Track_release_date', 'Track_pop', 'Artist_pop']]).toarray(), index=df.index, dtype=int)
column_name = ohe2.get_feature_names_out(['Track_release_date', 'Track_pop', 'Artist_pop'])
dummies.columns=column_name
df=pd.concat([df.drop(['Track_release_date', 'Track_pop', 'Artist_pop'], axis=1), dummies], axis=1)
 """

" ohe2 = OneHotEncoder(categories=ohe.categories_,handle_unknown='ignore')\ndummies = pd.DataFrame(ohe2.fit_transform(df[['Track_release_date', 'Track_pop', 'Artist_pop']]).toarray(), index=df.index, dtype=int)\ncolumn_name = ohe2.get_feature_names_out(['Track_release_date', 'Track_pop', 'Artist_pop'])\ndummies.columns=column_name\ndf=pd.concat([df.drop(['Track_release_date', 'Track_pop', 'Artist_pop'], axis=1), dummies], axis=1)\n "

In [77]:
# test.drop(['Track_release_date', 'Track_pop', 'Artist_pop'], axis='columns', inplace = True)  

In [78]:
#df.info(memory_usage = "deep")

In [79]:
#test.loc[:,test.columns.str.startswith('genre')]=test.loc[:,test.columns.str.startswith('genre')].astype('bool')
#df.loc[:,df.columns.str.startswith('genre')]=df.loc[:,df.columns.str.startswith('genre')].astype('bool')


In [80]:
sc=MinMaxScaler()
df.iloc[:,3:19]=sc.fit_transform(df.iloc[:,3:19])
pickle.dump(sc, open('data/sc.sav', 'wb'))

In [81]:
test.iloc[:,3:19]=sc.transform(test.iloc[:,3:19])

In [82]:
playvec=pd.DataFrame(test.sum(axis=0)).T
playvec

,track_uri,artist_uri,album_uri,danceability,energy,key,loudness,mode,speechiness,acousticness,...,valence,tempo,duration_ms,time_signature,Track_release_date,Track_pop,Artist_pop,genre|album_rock,genre|hard_rock,genre|rock
0,7e89621JPkKaeDSTQ3avtg7ACxUo21jtTHzy7ZEV56vU1h...,4MVyzYMgTwdP7Z49wAZHx06ZLTlhejhndI4Rh53vYhrY7b...,54V1ljNtyzAm053oJqi0SH4qUMByJ3Pk94BFnCmGaUPS0d...,54.9172,83.865234,47.272729,82.857875,70.0,7.98893,6.343812,...,64.644531,50.142073,4.65217,80.000001,98.275002,81.822113,71.45,20.265625,24.171875,65.5625


In [83]:
playvec.columns

Index(['track_uri', 'artist_uri', 'album_uri', 'danceability', 'energy', 'key',
       'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness',
       'liveness', 'valence', 'tempo', 'duration_ms', 'time_signature',
       'Track_release_date', 'Track_pop', 'Artist_pop', 'genre|album_rock',
       'genre|hard_rock', 'genre|rock'],
      dtype='object')

In [84]:
df.columns

Index(['track_uri', 'artist_uri', 'album_uri', 'danceability', 'energy', 'key',
       'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness',
       'liveness', 'valence', 'tempo', 'duration_ms', 'time_signature',
       'Track_release_date', 'Track_pop', 'Artist_pop', 'genre|album_rock',
       'genre|hard_rock', 'genre|rock'],
      dtype='object')

In [85]:
test.columns

Index(['track_uri', 'artist_uri', 'album_uri', 'danceability', 'energy', 'key',
       'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness',
       'liveness', 'valence', 'tempo', 'duration_ms', 'time_signature',
       'Track_release_date', 'Track_pop', 'Artist_pop', 'genre|album_rock',
       'genre|hard_rock', 'genre|rock'],
      dtype='object')

In [86]:
df.head()

,track_uri,artist_uri,album_uri,danceability,energy,key,loudness,mode,speechiness,acousticness,...,valence,tempo,duration_ms,time_signature,Track_release_date,Track_pop,Artist_pop,genre|album_rock,genre|hard_rock,genre|rock
0,3uvsVUrAaGQJCTEUR1S3Sx,0ypTT9UqAU5sZpPo5JZmjR,2kIoMj5Ht14l2PnNRa2abC,0.529412,0.422119,0.272727,0.766890,0.0,0.027976,0.786275,...,0.145020,0.519760,0.048079,0.8,1.0,0.000000,0.350000,0.0,0.0,0.0
1,3uvsVUrAaGQJCTEUR1S3Sx,0ypTT9UqAU5sZpPo5JZmjR,2kIoMj5Ht14l2PnNRa2abC,0.529412,0.422119,0.272727,0.766890,0.0,0.027976,0.786275,...,0.145020,0.519760,0.048079,0.8,1.0,0.000000,0.350000,0.0,0.0,0.0
2,4JxNnZ2wf9t3KGnB0cIwRU,0ypTT9UqAU5sZpPo5JZmjR,0CMDyjTSb2N8kpk3fSbiUp,0.530400,0.341064,0.818182,0.750830,0.0,0.032341,0.573039,...,0.295898,0.320160,0.053098,0.8,1.0,0.000000,0.350000,0.0,0.0,0.0
3,74Iy3ovRFMr5erVPha9SAa,0ypTT9UqAU5sZpPo5JZmjR,0gbEBO45KjT4pAlB83NRu4,0.426100,0.504883,0.545455,0.781501,0.0,0.045610,0.046875,...,0.131958,0.539770,0.045918,0.8,1.0,0.000000,0.350000,0.0,0.0,0.0
4,0heE5tAAaDQmnGhVDImPl2,0h3YCmvRJ2jqt4jFiR6nGL,3g7TTE6375PGIBsM9Tlk9I,0.499011,0.594238,0.727273,0.859325,0.0,0.039189,0.257108,...,0.303955,0.495248,0.044765,0.8,1.0,0.670407,0.530078,0.0,0.0,0.0


In [87]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300465 entries, 0 to 300464
Data columns (total 22 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   track_uri           300465 non-null  object 
 1   artist_uri          300465 non-null  object 
 2   album_uri           300465 non-null  object 
 3   danceability        300465 non-null  float64
 4   energy              300465 non-null  float64
 5   key                 300465 non-null  float64
 6   loudness            300465 non-null  float64
 7   mode                300465 non-null  float64
 8   speechiness         300465 non-null  float64
 9   acousticness        300465 non-null  float64
 10  instrumentalness    300465 non-null  float64
 11  liveness            300465 non-null  float64
 12  valence             300465 non-null  float64
 13  tempo               300465 non-null  float64
 14  duration_ms         300465 non-null  float64
 15  time_signature      300465 non-nul

In [88]:
playvec.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 22 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   track_uri           1 non-null      object
 1   artist_uri          1 non-null      object
 2   album_uri           1 non-null      object
 3   danceability        1 non-null      object
 4   energy              1 non-null      object
 5   key                 1 non-null      object
 6   loudness            1 non-null      object
 7   mode                1 non-null      object
 8   speechiness         1 non-null      object
 9   acousticness        1 non-null      object
 10  instrumentalness    1 non-null      object
 11  liveness            1 non-null      object
 12  valence             1 non-null      object
 13  tempo               1 non-null      object
 14  duration_ms         1 non-null      object
 15  time_signature      1 non-null      object
 16  Track_release_date  1 non-null

In [89]:
df['sim']=cosine_similarity(df.drop(['track_uri', 'artist_uri', 'album_uri'], axis = 1),playvec.drop(['track_uri', 'artist_uri', 'album_uri'], axis = 1))
df['sim2']=cosine_similarity(df.iloc[:,16:-1],playvec.iloc[:,16:])
df['sim3']=cosine_similarity(df.iloc[:,19:-2],playvec.iloc[:,19:])
df = df.sort_values(['sim3','sim2','sim'],ascending = False,kind='stable')
qq=df.groupby('artist_uri').head(5).track_uri.head(50)     #to limit recmmendation by same artist
aa=sp.tracks(qq[0:50])
Fresult=pd.DataFrame()
for i in range(50):
    result=pd.DataFrame([i])
    result['track_name']=aa['tracks'][i]['name']
    result['artist_name']=aa['tracks'][i]['artists'][0]['name']
    #result['url']=aa['tracks'][i]['external_urls']['spotify']
    #result['image']=aa['tracks'][i]['album']['images'][1]['url']
    Fresult=pd.concat([Fresult,result],axis=0)
Fresult

,0,track_name,artist_name
0,0,Numb,Linkin Park
0,1,Stressed Out,Twenty One Pilots
0,2,Stressed Out,Twenty One Pilots
0,3,Ride,Twenty One Pilots
0,4,Ride,Twenty One Pilots
0,5,Get Lucky (Radio Edit) [feat. Pharrell William...,Daft Punk
0,6,Get Lucky (Radio Edit) [feat. Pharrell William...,Daft Punk
0,7,"New Person, Same Old Mistakes",Tame Impala
0,8,505,Arctic Monkeys
0,9,Heathens,Twenty One Pilots


In [90]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 300465 entries, 11469 to 164162
Data columns (total 25 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   track_uri           300465 non-null  object 
 1   artist_uri          300465 non-null  object 
 2   album_uri           300465 non-null  object 
 3   danceability        300465 non-null  float64
 4   energy              300465 non-null  float64
 5   key                 300465 non-null  float64
 6   loudness            300465 non-null  float64
 7   mode                300465 non-null  float64
 8   speechiness         300465 non-null  float64
 9   acousticness        300465 non-null  float64
 10  instrumentalness    300465 non-null  float64
 11  liveness            300465 non-null  float64
 12  valence             300465 non-null  float64
 13  tempo               300465 non-null  float64
 14  duration_ms         300465 non-null  float64
 15  time_signature      300465 non

In [91]:
playvec.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 22 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   track_uri           1 non-null      object
 1   artist_uri          1 non-null      object
 2   album_uri           1 non-null      object
 3   danceability        1 non-null      object
 4   energy              1 non-null      object
 5   key                 1 non-null      object
 6   loudness            1 non-null      object
 7   mode                1 non-null      object
 8   speechiness         1 non-null      object
 9   acousticness        1 non-null      object
 10  instrumentalness    1 non-null      object
 11  liveness            1 non-null      object
 12  valence             1 non-null      object
 13  tempo               1 non-null      object
 14  duration_ms         1 non-null      object
 15  time_signature      1 non-null      object
 16  Track_release_date  1 non-null

In [93]:
df['sim']=cosine_similarity(df.iloc[:,3:16],playvec.iloc[:,3:16])
df['sim2']=cosine_similarity(df.loc[:, df.columns.str.startswith('T')|df.columns.str.startswith('A')],playvec.loc[:, playvec.columns.str.startswith('T')|playvec.columns.str.startswith('A')])
df['sim3']=cosine_similarity(df.loc[:, df.columns.str.startswith('genre')],playvec.loc[:, playvec.columns.str.startswith('genre')])
df['sim4']=(df['sim']+df['sim2']+df['sim3'])/3
df = df.sort_values(['sim4'],ascending = False,kind='stable')
# genra>audio>pop
qq=df.groupby('artist_uri').head(5).track_uri.head(50)
aa=sp.tracks(qq[0:50])
Fresult=pd.DataFrame()
for i in range(50):
    result=pd.DataFrame([i])
    result['track_name']=aa['tracks'][i]['name']
    result['artist_name']=aa['tracks'][i]['artists'][0]['name']
    #result['url']=aa['tracks'][i]['external_urls']['spotify']
    #result['image']=aa['tracks'][i]['album']['images'][1]['url']
    Fresult=pd.concat([Fresult,result],axis=0)
Fresult

,0,track_name,artist_name
0,0,Supermassive Black Hole,Muse
0,1,Still into You,Paramore
0,2,Parklife - 2012 Remaster,Blur
0,3,Houdini,Foster The People
0,4,Heaven Knows I'm Miserable Now - 2011 Remaster,The Smiths
0,5,Kyouran Hey Kids!!,THE ORAL CIGARETTES
0,6,Violent Pornography,System Of A Down
0,7,Cold Cold Cold,Cage The Elephant
0,8,"A Little Less Sixteen Candles, A Little More ""...",Fall Out Boy
0,9,(Can't Get My) Head Around You,The Offspring


In [94]:
Spotifyresult=pd.DataFrame()
for i in range(len(test)-1):
    if len(Spotifyresult)>=50:
        break
    ff=sp.recommendations(seed_tracks=list(test.track_uri[1+i:5+i]),limit=2)
    for z in range(2):
        result=pd.DataFrame([z+(2*i)+1])
        result['track_name']=ff['tracks'][z]['name']
        result['artist_name']=ff['tracks'][z]['artists'][0]['name']
        #result['uri']=ff['tracks'][z]['id']
        #result['url']=ff['tracks'][z]['external_urls']['spotify']
        #result['image']=ff['tracks'][z]['album']['images'][1]['url']
        Spotifyresult=pd.concat([Spotifyresult,result],axis=0)
Spotifyresult

,0,track_name,artist_name
0,1,Cardiac Arrest,Bad Suns
0,2,Feed My Frankenstein,Alice Cooper
0,3,Lotus Eater,Foster The People
0,4,I Got the Six - 2008 Remaster,ZZ Top
0,5,Say It Ain't So,Weezer
0,6,Friday I'm In Love,The Cure
0,7,Californication,Red Hot Chili Peppers
0,8,Punchin' Bag,Cage The Elephant
0,9,Midlife Crisis,Faith No More
0,10,Awake,Godsmack


sorry ur playlist must have atleast 5 tracks for this method to work